In [1]:
import requests
import pandas as pd
import numpy as np
!pip install folium
import folium 

In [2]:
CLIENT_ID = 'M0DXZUDLZA4G4VDQYMBXXHKKYTIVG0V5P5WSCFQOI5PDRZOT' # your Foursquare ID
CLIENT_SECRET = 'BTOLGCB3G5JZH5LKS23NBQ0JELZK5VNOMZA0DH2Q4V0FS5WI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M0DXZUDLZA4G4VDQYMBXXHKKYTIVG0V5P5WSCFQOI5PDRZOT
CLIENT_SECRET:BTOLGCB3G5JZH5LKS23NBQ0JELZK5VNOMZA0DH2Q4V0FS5WI


In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Moscow",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Moscow', 'Moscow')

In [6]:
d["suggestedBounds"], d["totalResults"]


({'ne': {'lat': 55.761899105903474, 'lng': 37.629018866729595},
  'sw': {'lat': 55.744218118766504, 'lng': 37.59819576534481}},
 66)

In [7]:
d["geocode"]

{'what': '',
 'where': 'moscow',
 'center': {'lat': 55.75222, 'lng': 37.61556},
 'displayString': 'Moscow, Russia',
 'cc': 'RU',
 'geometry': {'bounds': {'ne': {'lat': 56.019074988673324,
    'lng': 37.96493399950954},
   'sw': {'lat': 55.50402900005645, 'lng': 37.14263708584998}}},
 'slug': 'moscow',
 'longId': '72057594038452837'}

In [8]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '515fba1de4b0d9eb08328d90',
  'name': 'Louis Vuitton Lounge',
  'location': {'address': 'Красная пл., 3',
   'lat': 55.75507193409188,
   'lng': 37.619723812041876,
   'labeledLatLngs': [{'label': 'display',
     'lat': 55.75507193409188,
     'lng': 37.619723812041876}],
   'cc': 'RU',
   'city': 'Москва',
   'state': 'Москва',
   'country': 'Россия',
   'formattedAddress': ['Красная пл., 3', 'Москва', 'Россия']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-515fba1de4b0d9eb08328d90-0'}

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': 'Красная пл., 3', 'lat': 55.75507193409188, 'lng': 37.619723812041876, 'labeledLatLngs': [{'label': 'display', 'lat': 55.75507193409188, 'lng': 37.619723812041876}], 'cc': 'RU', 'city': 'Москва', 'state': 'Москва', 'country': 'Россия', 'formattedAddress': ['Красная пл., 3', 'Москва', 'Россия']}
{'address': 'ул. Охотный Ряд, 2', 'lat': 55.75722883551855, 'lng': 37.61658630778593, 'labeledLatLngs': [{'label': 'display', 'lat': 55.75722883551855, 'lng': 37.61658630778593}], 'postalCode': '127159', 'cc': 'RU', 'neighborhood': 'Китай-город', 'city': 'Москва', 'state': 'Москва', 'country': 'Россия', 'formattedAddress': ['ул. Охотный Ряд, 2', '127159, Москва', 'Россия']}
{'address': 'Никольская ул., 8/1', 'lat': 55.7572334350327, 'lng': 37.62268793049797, 'labeledLatLngs': [{'label': 'display', 'lat': 55.7572334350327, 'lng': 37.62268793049797}], 'postalCode': '109012', 'cc': 'RU', 'city': 'Москва', 'state': 'Москва', 'country': 'Россия', 'formattedAddress': ['Никольская ул., 8/1'

,uid,name,shortname,address,postalcode,lat,lng
0,4f3b52bfe4b0ce0258fcafd4,Starbucks,Coffee Shop,,127159,55.757229,37.616586
1,59d1ea22d4cc9807e348e1a9,Surf Coffee,Coffee Shop,,109012,55.757233,37.622688
2,5e30114d13749b00085e7900,Антипа,Coffee Shop,,119019,55.747743,37.605589
3,5e5e5e7195d041000803469e,Fine Coffee,Coffee Shop,,119019,55.747208,37.608055
4,505ec2cbe4b0a06084de3bb0,Кофемания,Coffee Shop,,109012,55.754636,37.621629


In [12]:
moscow_center = d["geocode"]["center"]
moscow_center

{'lat': 55.75222, 'lng': 37.61556}

In [14]:
from folium import plugins


map_moscow = folium.Map(location=[55.75222, 37.61556], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_moscow)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_moscow.add_child(plugins.HeatMap(hm_data))

map_moscow

In [16]:
lat = 55.75222
lng = 37.61556
map_moscow = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Here!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_moscow)
map_moscow